In [1]:
%cd ~  # 设置当前目录路径
!unzip data/data132692/VOCdataset.zip -d work/dataset  # 将自制数据集进行解压
!pip install paddlex==2.1.0 -i https://mirror.baidu.com/pypi/simple  # 下载paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 9.1 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1593868 sha256=6cc04c93055687a2451e9a476ae00578651f26d47b9c0835b5d2d88048849deb
  Stored in directory: /home/aistudio/.cache/pip/wheels/95/5f/20/9e2b2cfb8b2bfae5a5374e947511a47c8909e74aaf6d6d4464
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp37-c

In [2]:
!paddlex --split_dataset --format VOC --dataset_dir 'work/dataset/VOCdataset' --val_value 0.2 --test_value 0.1

[03-12 16:34:28 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --split_dataset --format VOC --dataset_dir work/dataset/VOCdataset --val_value 0.2 --test_value 0.1
[03-12 16:34:28 MainThread @utils.py:79] WRN paddlepaddle version: 2.3.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: FutureWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-pad

In [3]:
# 数据增强
import paddlex as pdx
from paddlex import transforms as T
train_transforms = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomDistort(),
    T.RandomExpand(),
    T.RandomCrop(),
    T.ResizeByShort(short_size=250, max_size=-1),
    T.Normalize()
])
eval_transforms = T.Compose([
    T.ResizeByShort(short_size=250, max_size=-1),
    T.Normalize()
])

# 加载数据集
train_dataset = pdx.datasets.VOCDetection(
                        data_dir=r'work/dataset/VOCdataset',
                        file_list=r'work/dataset/VOCdataset/train_list.txt',
                        label_list=r'work/dataset/VOCdataset/labels.txt',
                        transforms=train_transforms,
                        # shuffle=True
                        )
eval_dataset = pdx.datasets.VOCDetection(
                        data_dir=r'work/dataset/VOCdataset',
                        file_list=r'work/dataset/VOCdataset/val_list.txt',
                        label_list=r'work/dataset/VOCdataset/labels.txt',
                        transforms=eval_transforms)

[03-12 16:34:35 MainThread @utils.py:79] WRN paddlepaddle version: 2.3.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: FutureWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationW

2023-03-12 16:34:38 [INFO]	Starting to read file list from dataset...
2023-03-12 16:34:38 [INFO]	490 samples in file work/dataset/VOCdataset/train_list.txt, including 490 positive samples and 0 negative samples.
creating index...
index created!
2023-03-12 16:34:38 [INFO]	Starting to read file list from dataset...
2023-03-12 16:34:38 [INFO]	140 samples in file work/dataset/VOCdataset/val_list.txt, including 140 positive samples and 0 negative samples.
creating index...
index created!


In [4]:
# 模型训练  Faster RCNN
num_classes = len(train_dataset.labels)
print(num_classes)
model = pdx.det.FasterRCNN(num_classes=num_classes, backbone='ResNet50')
model.train(
    num_epochs=10,
    train_dataset=train_dataset,
    train_batch_size=1,
    eval_dataset=eval_dataset,
    save_interval_epochs=1,
    learning_rate=0.00025,
    # learning_rate=0.0001,
    # lr_decay_epochs = [3,6],
    # lr_decay_gamma = 0.1,
    metric='VOC',
    save_dir='output/faster_rcnn_r50_fpn',
    use_vdl=True)

1
2023-03-12 16:34:39 [INFO]	Downloading ResNet50_cos_pretrained.pdparams from https://paddledet.bj.bcebos.com/models/pretrained/ResNet50_cos_pretrained.pdparams


W0312 16:34:38.871008   205 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0312 16:34:38.876703   205 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
100%|██████████| 92063/92063 [00:05<00:00, 17697.11KB/s]


2023-03-12 16:34:44 [INFO]	Loading pretrained model from output/faster_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.pdparams
2023-03-12 16:34:44 [WARNING]	neck.fpn_inner_res2_sum_lateral.weight is not in pretrained model
2023-03-12 16:34:44 [WARNING]	neck.fpn_inner_res2_sum_lateral.bias is not in pretrained model
2023-03-12 16:34:44 [WARNING]	neck.fpn_res2_sum.weight is not in pretrained model
2023-03-12 16:34:44 [WARNING]	neck.fpn_res2_sum.bias is not in pretrained model
2023-03-12 16:34:44 [WARNING]	neck.fpn_inner_res3_sum_lateral.weight is not in pretrained model
2023-03-12 16:34:44 [WARNING]	neck.fpn_inner_res3_sum_lateral.bias is not in pretrained model
2023-03-12 16:34:44 [WARNING]	neck.fpn_res3_sum.weight is not in pretrained model
2023-03-12 16:34:44 [WARNING]	neck.fpn_res3_sum.bias is not in pretrained model
2023-03-12 16:34:44 [WARNING]	neck.fpn_inner_res4_sum_lateral.weight is not in pretrained model
2023-03-12 16:34:44 [WARNING]	neck.fpn_inner_res4_sum_lateral.bias is not 

In [5]:
# 在验证集上进行评估
import paddlex as pdx
model = pdx.load_model('output/faster_rcnn_r50_fpn/best_model')  # 加载模型
eval_result = model.evaluate(eval_dataset, batch_size=1, return_details=False)
# eval_result = model.evaluate(eval_dataset, batch_size=1, return_details=True)
print('eval result:\n',eval_result)

2023-03-12 16:52:46 [INFO]	Model[FasterRCNN] loaded.
2023-03-12 16:52:46 [INFO]	Start to evaluate(total_samples=140, total_steps=140)...
2023-03-12 16:53:01 [INFO]	Accumulating evaluatation results...
eval result:
 OrderedDict([('bbox_map', 94.98506011562284)])


In [7]:
# 模型预测
import paddlex as pdx
img = r'photo/12.jpg'
model = pdx.load_model('output/faster_rcnn_r50_fpn/best_model')
result = model.predict(img)
print("Predict Result: ", result)
pdx.det.visualize(img, result, save_dir='./result',threshold=0.6)

2023-03-12 16:57:36 [INFO]	Model[FasterRCNN] loaded.
Predict Result:  [{'category_id': 0, 'category': 'Person', 'bbox': [267.6619567871094, 265.86968994140625, 176.98269653320312, 502.552978515625], 'score': 0.9902663826942444}, {'category_id': 0, 'category': 'Person', 'bbox': [267.6175231933594, 276.1723937988281, 91.35528564453125, 507.5971374511719], 'score': 0.09246087074279785}]
2023-03-12 16:57:36 [INFO]	The visualized result is saved at ./result/visualize_12.jpg


In [8]:
# 转化为inference模型
!paddlex --export_inference --model_dir=output/faster_rcnn_r50_fpn/best_model --save_dir=output

[03-12 16:57:41 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --export_inference --model_dir=output/faster_rcnn_r50_fpn/best_model --save_dir=output
[03-12 16:57:41 MainThread @utils.py:79] WRN paddlepaddle version: 2.3.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: FutureWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/li

In [3]:
# 模型预测
import paddlex as pdx
img = r'photo/684.jpg'
model = pdx.load_model('output/inference_model')
result = model.predict(img)
print("Predict Result: ", result)
pdx.det.visualize(img, result, save_dir='./result',threshold=0.8)

2023-03-24 14:52:32 [INFO]	Model[FasterRCNN] loaded.
Predict Result:  [{'category_id': 0, 'category': 'Person', 'bbox': [201.72789001464844, 318.3688049316406, 120.42369079589844, 467.1354675292969], 'score': 0.9633339643478394}, {'category_id': 0, 'category': 'Person', 'bbox': [357.6348876953125, 295.7972106933594, 137.32931518554688, 505.8344421386719], 'score': 0.9691698551177979}, {'category_id': 0, 'category': 'Person', 'bbox': [250.74049377441406, 312.01397705078125, 123.40846252441406, 492.17193603515625], 'score': 0.2949315011501312}, {'category_id': 0, 'category': 'Person', 'bbox': [196.59852600097656, 340.4701232910156, 72.10856628417969, 418.8099060058594], 'score': 0.12481983751058578}, {'category_id': 0, 'category': 'Person', 'bbox': [216.89573669433594, 295.96002197265625, 287.16148376464844, 495.43499755859375], 'score': 0.8195303678512573}, {'category_id': 0, 'category': 'Person', 'bbox': [260.0248107910156, 319.1795959472656, 51.749908447265625, 431.9230651855469], 'sc